### 필요한 패키지 Import
DataFrame의 내용을 생략없이 출력하려면 아래 주석처리한 부분의 주석을 해제하고 실행

In [1]:
import os

import numpy as np
import pandas as pd

from collections import Counter

# pd.set_option('display.max_colwidth', None)

### 변수 정의

In [2]:
DATAPATH = "../../data/"
DATAFILE = os.path.join(DATAPATH, f"train.csv")
SOURCE_DATA = pd.read_csv(DATAFILE)
data = SOURCE_DATA.copy()

### Pandas 기본 기능을 이용하여 간단한 분석

In [3]:
data.head()

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화,3,IT과학,"{'annotators': ['08', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
1,ynat-v1_train_00001,어버이날 막따가 흐려저…남부지방 여튼 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
2,ynat-v1_train_00001,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화,3,생활문화,"{'annotators': ['14', '13', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
3,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회,2,IT과학,"{'annotators': ['14', '13', '10'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00
4,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,사회,2,IT과학,"{'annotators': ['16', '11', '15'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.28. 오전 9:54


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52530 entries, 0 to 52529
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        52530 non-null  object
 1   input_text                52530 non-null  object
 2   label_text                52530 non-null  object
 3   target                    52530 non-null  int64 
 4   predefined_news_category  52530 non-null  object
 5   annotations               52530 non-null  object
 6   url                       52530 non-null  object
 7   date                      52530 non-null  object
dtypes: int64(1), object(7)
memory usage: 3.2+ MB


In [5]:
data.describe()

,target
count,52530.000000
mean,3.048315
std,2.063432
min,0.000000
25%,1.000000
50%,3.000000
75%,5.000000
max,6.000000


In [6]:
data.label_text.unique()

array(['생활문화', '사회', 'IT과학', '스포츠', '세계', '정치', '경제'], dtype=object)

In [7]:
label_dict = {
    "정치": 0,
    "경제": 1,
    "사회": 2,
    "생활문화": 3,
    "세계": 4,
    "IT과학": 5,
    "스포츠": 6,
}

### Label Error Detection

1. 먼저 target과 label_text가 제대로 연결되지 않은 샘플이 존재하는지를 확인

In [9]:
mismatched_data = data[data["target"] != data["label_text"].map(label_dict)]
mismatched_data

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date


2. Annotation의 Voting 결과를 label이라고 생각했을 때, 제대로 연결되지 않은 샘플이 존재하는지 확인

- 예를 들어, 특정 데이터의 annotations column이 다음과 같다고 하면, first-scope의 최빈값인 "생활문화"를 label이라고 간주. \
만약, first-scope의 모든 값이 다르면, second-scope의 최빈값을 label이라고 생각한다.

```
    annotations = {
        "annotators": ["08", "13", "07"],
        "annotations": {
            "first-scope": ["생활문화", "생활문화", "IT과학"],
            "second-scope": ["IT과학", "해당없음", "해당없음"],
            "third-scope": ["해당없음", "해당없음", "해당없음"],
        },
    }

```

annotations 컬럼의 내용을 보면 dictionary의 형태를 띄지만, 실제 데이터 타입은 str이다. \
따라서 eval 함수를 사용하여 dictionary로써 동작하게 처리해준다. \
먼저, first-scope의 값이 모두 다른 경우가 있는지를 확인한다.

In [11]:
data = data.assign(
    annotations = data["annotations"].apply(eval),
)

In [21]:
for i in range(len(data)):
    first_scope_list = data.iloc[i]["annotations"]["annotations"]["first-scope"]
    if len(set(first_scope_list)) == 3:
        print(set(first_scope_list))
    # if len(set(first_scope_list)) == 2:
    #     print(set(first_scope_list))
    #     if i > 10:
    #         break

First-scope의 값이 모두 다른 경우가 없다는 것을 확인하였다. \
(주석처리한 부분을 해제하여 실행해보면 코드가 정상작동함을 확인할 수 있음) 

즉, annotations dictionary에서 first-scope만 살펴보아도 label을 결정할 수 있다.

### 함수를 정의하여 Label Error 발생 여부 확인

`voting_result` 변수에 데이터의 annotations 컬럼에 나타난 first-scope의 최빈값을 저장하고, \
그 값이 해당 데이터의 Label값과 일치하는지를 확인한다.

위에서 `target`과 `label_text`는 모두 제대로 연결되어 있음을 확인하였으니, 둘 중 하나와만 비교하면 될 것이다.

In [22]:
def check_label_error(data):
    voting_result = Counter(data.annotations["annotations"]["first-scope"]).most_common(1)[0][0]
    return voting_result != data.label_text

In [23]:
label_error_checked = data.copy()
label_error_checked["label_error"] = label_error_checked.apply(check_label_error, axis=1).apply(pd.Series)

In [24]:
label_error_checked[label_error_checked["label_error"] == True]

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date,label_error
54,ynat-v1_train_00049,반민특위 임무 아직 끝나지 않았다,경제,1,생활문화,"{'annotators': ['11', '07', '16'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.03. 오후 4:11,True
120,ynat-v1_train_00109,서울에 다시 오존주의보…도심·서북·동북권 발령종합,IT과학,5,생활문화,"{'annotators': ['03', '14', '13'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.07.27. 오후 3:41,True
281,ynat-v1_train_00250,트럼프·엡스타인 1992년 마러라고서 여성 20여명과 파티,정치,0,세계,"{'annotators': ['09', '18', '11'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.07.11. 오전 6:18,True
362,ynat-v1_train_00316,예스24 우리가 사랑한 24인의 작가들 부산서 전시,경제,1,생활문화,"{'annotators': ['02', '15', '12'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.08. 오전 10:48,True
393,ynat-v1_train_00341,현대모비스 3연속 100점대 대승…삼성 만난 라건아 39점,사회,2,스포츠,"{'annotators': ['18', '02', '03'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.10.19 21:53,True
...,...,...,...,...,...,...,...,...,...
52218,ynat-v1_train_45399,1승 11패 한국 여자배구 VNL 5주차서 벼랑 끝...,정치,0,스포츠,"{'annotators': ['07', '02', '10'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2019.06.16 06:44,True
52262,ynat-v1_train_45436,美 수배받은 콜롬비아 최대 반군 전 사령관 의원으로 취임,경제,1,세계,"{'annotators': ['02', '13', '09'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.12. 오전 4:51,True
52302,ynat-v1_train_45472,11월 전국 주택 매매가 상승폭 둔화…일부 광역시는 강세,스포츠,6,경제,"{'annotators': ['07', '02', '13'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.12.03. 오전 11:49,True
52373,ynat-v1_train_45535,수원 아시아 4강 이끈 특급 선방 신화용 분석·조언의 힘,경제,1,스포츠,"{'annotators': ['11', '14', '10'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.09.19 22:38,True


AI Stages의 학습데이터 설명을 보면 다음과 같이 나와 있다.
> 노이즈 데이터의 경우 전체 학습 데이터의 15%(총 6,852개)에 해당합니다. noise data 중 80%(5,481개)는 G2P를 이용한 text perturbation으로 생성되었으며, prescriptive pronunciation과 descriptive pronunciation을 1:1 비율로 사용하였 noise를 생성하였습니다. 나머지 20%(1,371개)는 labeling 오류로 만들어진 데이터 입니다.

실제로 `annotations`의 최빈값과 `label_text`가 다른 데이터의 개수가 1371개임을 위에서 확인하였다. \
따라서 label error에 대한 데이터는 모두 검출한 셈이다.

이제 label error를 갖는 데이터를 모두 교정해주어야 한다.

###  Error Revise 수행

In [35]:
label_error_checked[label_error_checked["input_text"] == label_error_checked.iloc[54]["input_text"]]

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date,label_error
53,ynat-v1_train_00049,반민특위 임무 아직 끝나지 않았다,사회,2,생활문화,"{'annotators': ['11', '07', '16'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.03. 오후 4:11,False
54,ynat-v1_train_00049,반민특위 임무 아직 끝나지 않았다,경제,1,생활문화,"{'annotators': ['11', '07', '16'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.03. 오후 4:11,True


In [30]:
def revise_label_error(data):
    voting_result = Counter(data["annotations"]["first-scope"]).most_common(1)[0][0]
    return voting_result

In [37]:
label_error_revised = data.copy()
label_error_revised["label_text"] = data["annotations"].apply(revise_label_error)
label_error_revised["target"] = label_error_revised["label_text"].map(label_dict)

참고로, label error에 의한 증강을 적용한 것이므로, 이를 교정하면 중복되는 아래와 같이 데이터가 존재하게 되는 것이다. \
따라서 이에 대한 처리는 별도로 해주어야 한다.

In [38]:
label_error_revised[label_error_checked["input_text"] == label_error_revised.iloc[54]["input_text"]]

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
53,ynat-v1_train_00049,반민특위 임무 아직 끝나지 않았다,사회,2,생활문화,"{'annotators': ['11', '07', '16'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.03. 오후 4:11
54,ynat-v1_train_00049,반민특위 임무 아직 끝나지 않았다,사회,2,생활문화,"{'annotators': ['11', '07', '16'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.03. 오후 4:11


In [39]:
label_error_revised.iloc[53] == label_error_revised.iloc[54]

ID                          True
input_text                  True
label_text                  True
target                      True
predefined_news_category    True
annotations                 True
url                         True
date                        True
dtype: bool

### Label Error가 수정되었는지 최종 확인
Label error revision이 정상적으로 수행되었는지, 다시 한 번 label error를 확인한다. \
아래 코드를 실행하면 빈 데이터프레임이 출력되므로 정상적으로 수행되었음을 알 수 있다.

In [40]:
label_error_checked = label_error_revised.copy()
label_error_checked["label_error"] = label_error_checked.apply(check_label_error, axis=1).apply(pd.Series)
label_error_checked[label_error_checked["label_error"] == True]

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date,label_error


### Label Error를 수정한 csv 파일을 저장

중복 데이터에 대한 처리는 별도로 하고, 우선 레이블 에러만을 교정한 파일을 따로 저장해준다.

In [41]:
label_error_revised.to_csv("../../data/preprocessed/train_label_error_revised.csv", index=False)